In [3]:
#!/usr/bin/env python3
from Crypto.Util.number import long_to_bytes
from Crypto.Random import get_random_bytes
from Crypto.Hash import SHA256
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

import rsa

import telnetlib
import json

BLOCK_SIZE = 16

# Change this to REMOTE = False if you are running against a local instance of the server
REMOTE = True

# Remember to change the port if you are re-using this client for other challenges
PORT = 50800

if REMOTE:
    host = "aclabs.ethz.ch"
else:
    host = "localhost"

tn = telnetlib.Telnet(host, PORT)

def readline():
    return tn.read_until(b"\n")

def json_recv():
    line = readline()
    return json.loads(line.decode())

def json_send(req):
    request = json.dumps(req).encode()
    tn.write(request + b"\n")

def xor(a: bytes, b: bytes) -> bytes:
    return bytes(x ^ y for x, y in zip(a, b))   

def blockify(a):
    return [a[i : i + BLOCK_SIZE] for i in range(0, len(a), BLOCK_SIZE)]


In [4]:
pk, sk, primes = rsa.rsa_key_gen()

(N, e) = pk
(N, d) = sk
(p, q) = primes

json_send({
    'command': 'set_parameters',
    'N': N,
    'e': e,
    'd': d,
    'p': p,
    'q': q
})

res = json_recv()

print(res)

json_send({
    'command': 'encrypted_flag'
})

res = json_recv()
print(res)

flag = res['res'].split(':')
c = int(flag[1])

print('flag in ptxt: ', long_to_bytes(rsa.rsa_dec(sk, c)))

{'res': 'RSA Parameters were successfully updated'}
{'res': 'Here is your flag... oh no, it is RSA encrypted: 14084993352119985341877569799882321410323274424039371572324637586732484359789903142429948674059262756120484694195517732266493626952411016677959290026794729164944621771529815671373923160164703236837001339556319516728809931791060484160209353925143932115702005949293813832199188534855976391098768657568454447836653200039434631019248902162482755436417692529444539079603727423074212372479929050800544376816590468555767665765097573365164651201968493900545006983989026518191322900684272364165314946518936816225611660564213384346229789278225995425389034721427840585386225957159536780624337741901629804394446208059696471634176'}
b'flag{d0_not_under_4ny_c1rcumstances_us3_textb00k_rsa}'
